In [ ]:
import os

import pyspark.sql.functions as F  # noqa
import pyspark.sql.types as T  # noqa
from pyspark.sql import SparkSession

os.environ["AWS_PROFILE"] = "blueriver"

CATALOG = "glue_catalog"
ICEBERG_S3_ROOT_PATH = "s3a://blueriver-datalake/iceberg"

spark = (
    SparkSession.builder.appName("")
    .config("spark.sql.defaultCatalog", CATALOG)
    .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
    .config(f"spark.sql.catalog.{CATALOG}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
    .config(f"spark.sql.catalog.{CATALOG}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config(f"spark.sql.catalog.{CATALOG}.warehouse", ICEBERG_S3_ROOT_PATH)
    .config(f"spark.sql.catalog.{CATALOG}.s3.path-style-access", True)
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "software.amazon.awssdk.auth.credentials.DefaultCredentialsProvider",
    )
    .config("spark.sql.caseSensitive", True)
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
    .config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
    .config("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
    .getOrCreate()
)

In [ ]:
df = spark.read.parquet("s3a://blueriver-datalake/data/raw_rds/store/tb_lower", recursiveFileLookup=True)
df.show(truncate=False)